In [1]:
import os
import pandas as pd
from pathlib import Path

In [ ]:
os.chdir(r"C:\Users\Rand Sobczak Jr\_rts\mlops\mimiccds")


In [5]:
hosp_dir = Path(r"C:\Users\Rand Sobczak Jr\_rts\mlops\mimiccds/real_mimic/data/mimiciv")

In [6]:
# Load data
base_path = Path("../../real_mimic/data/mimiciv")
patients = pd.read_csv(hosp_dir / "_hosp" / "patients.csv")
admissions = pd.read_csv(hosp_dir / "_hosp" / "admissions.csv")


In [7]:
# Merge and label
df = admissions.merge(patients, on="subject_id", how="left")
df['dod'] = pd.to_datetime(df['dod'], errors='coerce')
df['dischtime'] = pd.to_datetime(df['dischtime'], errors='coerce')
df['mortality_30d'] = (
    (df['dod'].notna()) &
    ((df['dod'] - df['dischtime']).dt.days <= 30) &
    ((df['dod'] - df['dischtime']).dt.days >= 0)
).astype(int)



In [8]:
# Sort and filter to index admission
df = df.sort_values("admittime").drop_duplicates("subject_id", keep="first")

In [9]:
# Feature engineering
df['gender'] = df['gender'].map({'M': 0, 'F': 1})
df = df[['subject_id', 'anchor_age', 'gender', 'mortality_30d']].dropna()


In [12]:
# Save features and labels
output_path = Path(r"C:\Users\Rand Sobczak Jr\_rts\mlops\mimiccds\real_mimic\output")
output_path.mkdir(parents=True, exist_ok=True)
df.drop(columns=['mortality_30d']).to_csv(output_path / "X_mortality.csv", index=False)
df[['subject_id', 'mortality_30d']].to_csv(output_path / "y_mortality.csv", index=False)


In [13]:
print("✅ Saved X_mortality.csv and y_mortality.csv to:", output_path)

✅ Saved X_mortality.csv and y_mortality.csv to: C:\Users\Rand Sobczak Jr\_rts\mlops\mimiccds\real_mimic\output
